<a href="https://colab.research.google.com/github/nuvita97/project-quotation/blob/main/export_pq_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

uploaded = files.upload()

# Get the filename
filename = next(iter(uploaded))

# Read excel file in pd
table = pd.read_excel(filename)
table

# Choose needed fields
df = table.loc[:,['WBS ID', 'Task name', 'Description', 'Quantity', 'Unit sales price', 'Total sales price']]

# Remove NaN
df = df.fillna("")
df

# Get data of total price  
sum = df.loc[0]['Total sales price']

# Add 3 last rows to df (round number)
lastrow1 = {'WBS ID': '', 'Task name': '', 'Description': 'Tổng giá trị tiền hàng:', 'Quantity': '', 'Unit sales price': '', 'Total sales price': sum}
lastrow2 = {'WBS ID': '', 'Task name': '', 'Description': 'Thuế suất GTGT 10%:', 'Quantity': '', 'Unit sales price': '', 'Total sales price': round(sum*10/100, 0)}
lastrow3 = {'WBS ID': '', 'Task name': '', 'Description': 'Tổng giá trị thanh toán:', 'Quantity': '', 'Unit sales price': '', 'Total sales price': round(sum*110/100, 0)}

df = df.append(lastrow1, ignore_index=True)
df = df.append(lastrow2, ignore_index=True)
df = df.append(lastrow3, ignore_index=True)

# Delete first row
df = df.drop(0)

df

# Convert to lists
task_name = df['Task name'].to_list()
description = df['Description'].to_list()
quantity = df['Quantity'].to_list()
unit_sales_price = df['Unit sales price'].to_list()
total_sales_price = df['Total sales price'].to_list()

# Read Lookup file in pd
df2 = pd.read_excel('project-quotation/lookup_full_items.xlsx').fillna('')
df2.head()

# Convert columns in Lookup file to Lists
name = df2['name'].to_list()
name_vn = df2['name_vn'].to_list()
unit_vn = df2['unit_vn'].to_list()

# Merge Task name into Description
for i in range(len(description)):
  if description[i] == '':
    description[i] = task_name[i]

description

# Fix Quantity & Unit sales price
for i in range(len(description)):
  if '=' and ('0x' or '0 x') in description[i]:
    equal_id = description[i].index('=')
    added = description[i][equal_id:].replace(' ','')
    x_id = added.index('0x')
    quantity_new = added[x_id+2 :]

    quantity[i] = int(quantity_new)
    unit_sales_price[i] = total_sales_price[i] / quantity[i]
    description[i] = description[i][: equal_id]

description

# Loop & Create new list for Description
description_list = []

for item in description:
  for i in range(len(name)):
    if len(name_vn[i]) != 0:
      item = item.replace( name[i], name_vn[i] )
  description_list.append(item)

description_list

# Loop & Create new list for Unit 
unit_list = []
item_unit = ''

for item in description:
  for i in range(len(name)):
    if name[i] in item:
      if len(unit_vn[i]) != 0:
        item_unit = unit_vn[i]
        break
    else:
      item_unit = ''
  unit_list.append(item_unit)

unit_list

# Duplicate original df to df_final
df_final = df.copy()

# Assign lists to df_final
df_final['Task name'] = description_list
df_final['Description'] = unit_list
df_final['Quantity'] = quantity
df_final['Unit sales price'] = unit_sales_price

# Change 2 new column name
df_final = df_final.rename(columns={'Description': 'Unit'})
df_final = df_final.rename(columns={'Task name': 'Description'})

# Insert 5 blank rows
df_final.insert(2, '1', '')
df_final.insert(2, '2', '')
df_final.insert(2, '3', '')
df_final.insert(2, '4', '')
df_final.insert(2, '5', '')

df_final

#Use library dataframe_to_rows
rows = dataframe_to_rows(df_final, index=False, header=False)

wb = load_workbook('project-quotation/template_baogia_full.xlsx')
ws = wb.worksheets[0]

# Apply Borders to cells
thin_border = Border(top = Side(border_style='thin', color='FF000000'),    
                    right = Side(border_style='thin', color='FF000000'), 
                    bottom = Side(border_style='thin', color='FF000000'),
                    left = Side(border_style='thin', color='FF000000'))

# Copy from final table to Excel Template
for r_idx, row in enumerate(rows, 1):
  for c_idx, value in enumerate(row, 1):
    ws.cell(row=r_idx + 1, column=c_idx, value=value).border=thin_border

# Add new name for created file
new_filename = filename[:-5] + '_final.xlsx'

wb.save(new_filename)

In [ ]:
# from google.colab import files
# import pandas as pd
# from openpyxl import Workbook, load_workbook
# from openpyxl.styles import Border, Side
# from openpyxl.utils.dataframe import dataframe_to_rows

# uploaded = files.upload()

# # Get the filename
# filename = next(iter(uploaded))

# # Read excel file in pd
# table = pd.read_excel(filename)
# table

# # Choose needed fields
# df = table.loc[:,['WBS ID', 'Task name', 'Description', 'Quantity', 'Unit sales price', 'Total sales price']]
# df

# # Remove NaN
# df = df.fillna("")
# df

# # Convert to lists
# task_name = df['Task name'].to_list()
# description = df['Description'].to_list()

# # Read Lookup file in pd
# df2 = pd.read_excel('project-quotation/lookup_full.xlsx').fillna('')
# df2

# # Convert columns in Lookup file to Lists
# name = df2['name'].to_list()
# name_vn = df2['name_vn'].to_list()
# unit_vn = df2['unit_vn'].to_list()

# # Loop & Create new list for Description
# description_list = []

# for item in description:
#   for i in range(len(name)):
#     if len(name_vn[i]) != 0:
#       item = item.replace( name[i], name_vn[i] )
#   description_list.append(item)

# description_list

# # Loop & Create new list for Task name
# task_name_list = []

# for item in task_name:
#   for i in range(len(name)):
#     if len(name_vn[i]) != 0:
#       item = item.replace( name[i], name_vn[i] )
#   task_name_list.append(item)

# task_name_list

# # Loop & Create new list for Unit 

# unit_list = []
# item_unit = ''

# for item in description:
#   for i in range(len(name)):
#     if name[i] in item:
#       if len(unit_vn[i]) != 0:
#         item_unit = unit_vn[i]
#         break
#     else:
#       item_unit = ''
#   unit_list.append(item_unit)

# unit_list

# # Merge Task name into Description

# name_final = description_list

# for i in range(len(name_final)):
#   if name_final[i] == '' :
#     name_final[i] = task_name_list[i]

# name_final

# # Duplicate original df to df_final
# df_final = df.copy()

# # Assign lists to df_final
# df_final['Task name'] = name_final
# df_final['Description'] = unit_list

# # Change 2 new column name
# df_final = df_final.rename(columns={'Description': 'Unit'})
# df_final = df_final.rename(columns={'Task name': 'Description'})

# # Insert 5 blank rows
# df_final.insert(2, '1', '')
# df_final.insert(2, '2', '')
# df_final.insert(2, '3', '')
# df_final.insert(2, '4', '')
# df_final.insert(2, '5', '')

# df_final

# #Use library dataframe_to_rows
# rows = dataframe_to_rows(df_final, index=False, header=False)

# wb = load_workbook('project-quotation/template_baogia_full.xlsx')
# ws = wb.worksheets[0]

# # Apply Borders to cells
# thin_border = Border(top = Side(border_style='thin', color='FF000000'),    
#                     right = Side(border_style='thin', color='FF000000'), 
#                     bottom = Side(border_style='thin', color='FF000000'),
#                     left = Side(border_style='thin', color='FF000000'))

# # Copy from final table to Excel Template
# for r_idx, row in enumerate(rows, 1):
#   for c_idx, value in enumerate(row, 1):
#     ws.cell(row=r_idx + 1, column=c_idx, value=value).border=thin_border

# # Add new name for created file
# new_filename = filename[:-5] + '_modified.xlsx'

# wb.save(new_filename)